In [1]:
import torch

# First tests

In [2]:
x = torch.Tensor(5, 3)

In [3]:
x.numpy()
x = torch.rand(5, 3)


False


In [6]:
y = torch.rand(5, 3)

In [10]:
z = x + y

In [15]:
result = torch.Tensor(5, 3)

In [16]:
result


 0.0000e+00  0.0000e+00 -1.1551e+34
 4.5654e-41 -1.1552e+34  4.5654e-41
 1.8201e-38  0.0000e+00  1.8201e-38
 0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00
[torch.FloatTensor of size 5x3]

In [17]:
torch.add(x, y, out=result)


 1.4813  1.6816  0.6330
 1.7845  1.7222  2.0990
 1.0631  1.5775  1.9205
 1.5674  1.7441  0.7219
 1.0984  0.8575  2.1481
[torch.FloatTensor of size 5x3]

In [18]:
result


 1.4813  1.6816  0.6330
 1.7845  1.7222  2.0990
 1.0631  1.5775  1.9205
 1.5674  1.7441  0.7219
 1.0984  0.8575  2.1481
[torch.FloatTensor of size 5x3]

In [19]:
x.add_(y)


 1.4813  1.6816  0.6330
 1.7845  1.7222  2.0990
 1.0631  1.5775  1.9205
 1.5674  1.7441  0.7219
 1.0984  0.8575  2.1481
[torch.FloatTensor of size 5x3]

In [20]:
a = torch.ones(5)

In [21]:
a


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

In [22]:
b = a.numpy()

In [23]:
b

array([ 1.,  1.,  1.,  1.,  1.], dtype=float32)

# Autograd

In [40]:
from torch.autograd import Variable, Function

In [41]:
x = Variable(torch.ones(2, 2), requires_grad=True)

In [43]:
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [7]:
y = x + 2

In [10]:
z = y * y * 3

In [11]:
out = z.mean()

In [12]:
out.backward()

In [17]:
z.grad

Variable containing:
 0  0
 0  0
[torch.FloatTensor of size 2x2]

In [29]:
x = torch.randn(3)

In [30]:
x = Variable(x, requires_grad=True)

In [31]:
y = x * 2

In [32]:
while y.data.norm() < 1000:
    y = y * 2

In [33]:
y

Variable containing:
  541.2933
 1351.7925
-1334.0048
[torch.FloatTensor of size 3]

In [34]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

In [38]:
x.grad

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]

In [114]:
x = Variable(torch.randn(3), requires_grad=True)

In [113]:
out

Variable containing:
 0.6825
[torch.FloatTensor of size 1]

In [122]:
out

Variable containing:
 0.5096
[torch.FloatTensor of size 1]

# Define the network

In [86]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [87]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [99]:
input = Variable(torch.randn(1, 1, 32, 32))
print(input)
out = net(input)
out

Variable containing:
(0 ,0 ,.,.) = 
  0.0518 -0.5330 -1.3247  ...   0.5384  2.2835 -0.3841
 -0.1076  0.1956 -0.2280  ...   0.4589  0.2284 -0.7384
 -1.3266  1.0456  1.5442  ...   0.7847 -1.6906  0.8341
           ...             ⋱             ...          
 -1.5476  1.2849 -0.2031  ...   2.4626  0.2523 -0.9875
  0.0307  2.0446 -0.1591  ...   0.3435  0.3006  0.2261
 -0.9163  0.7763  0.8550  ...   0.5846  0.1897 -0.0504
[torch.FloatTensor of size 1x1x32x32]



Variable containing:
-0.0117  0.0188  0.0057 -0.0276 -0.0222 -0.0551  0.1195  0.0843  0.0960 -0.0513
[torch.FloatTensor of size 1x10]

In [89]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [90]:
print(input)

Variable containing:
(0 ,0 ,.,.) = 
 -0.7327 -0.6879  1.6376  ...  -0.2419 -0.0581 -0.4710
 -0.7368 -0.0121 -0.8629  ...   1.4172 -0.5414  1.0258
  1.0262  0.4058 -0.7109  ...   0.0746  0.3194 -0.7518
           ...             ⋱             ...          
  1.8159 -1.4942 -0.1338  ...  -0.3455 -0.1381 -0.1138
  0.2996  1.1363 -2.9362  ...  -0.9800  0.5413  0.6278
  1.0925 -1.2201 -0.5628  ...  -0.4546 -0.9694 -0.4620
[torch.FloatTensor of size 1x1x32x32]



In [94]:
output = net(input)
target = Variable(torch.range(1, 10))
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.5020
[torch.FloatTensor of size 1]



In [82]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  4.7394
 -1.1251
  7.5514
 -1.5325
 -4.9454
  2.5391
[torch.FloatTensor of size 6]



In [83]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [92]:
import torch.optim as optim

In [126]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()


In [98]:
import torch
import torchvision
import torchvision.transforms as transforms

ImportError: No module named torchvision